## 7. Concurrency and Parallelism

### 53 Use Threads for Blocking I/O, Avoid for Parallelism

In [1]:
def factorize(number):
    for i in range(1, number + 1):
        if number % i == 0:
            yield i

In [2]:
import time

In [3]:
numbers = [2139079, 1214759, 1516637, 1852285]

In [4]:
start = time.time()

for number in numbers:
    list(factorize(number))

end = time.time()
delta = end - start
print(f'Took {delta:.3f} seconds')

Took 0.456 seconds


In [5]:
from threading import Thread

In [6]:
class FactorizeThread(Thread):
    def __init__(self, number):
        super().__init__()
        self.number = number

    def run(self):
        self.factors = list(factorize(self.number))

In [7]:
start = time.time()

threads = []
for number in numbers:
    thread = FactorizeThread(number)
    thread.start()
    threads.append(thread)

for thread in threads:
    thread.join()

end = time.time()
delta = end - start
print(f'Took {delta:.3f} seconds')

Took 0.777 seconds


In [8]:
import select
import socket

def slow_systemcall():
    select.select([socket.socket()], [], [], 0.1)
    # isn't a time-consuming system call ?
    import time
    time.sleep(0.1)

In [9]:
start = time.time()

for _ in range(5):
    slow_systemcall()

end = time.time()
delta = end - start
print(f'Took {delta:.3f} seconds')

Took 0.503 seconds


In [10]:
start = time.time()

threads = []
for _ in range(5):
    thread = Thread(target=slow_systemcall)
    thread.start()
    threads.append(thread)

def compute_helicopter_location(index):
    pass

for i in range(5):
    compute_helicopter_location(i)

for thread in threads:
    thread.join()

end = time.time()
delta = end - start
print(f'Took {delta:.3f} seconds')

Took 0.105 seconds


> - 파이썬 스레드는 GIL(전역 인터프리터 락)로 인해 다중 CPU 코어에서 병렬로 실행될 수 없다.
> - GIL이 있음에도 불구하고 파이썬 스레드는 여전히 유용하다. 스레드를 사용하면 여러 일을 동시에 진행하는 프로그램을 쉽게 기술할 수 있기 때문이다.
> - 파이썬 스레드를 사용해 여러 시스템 콜을 병렬로 할 수 있다. 이를 활용하면 블로킹 I/O와 계산을 동시에 수행할 수 있다.